In [1]:
import ollama

In [2]:
model = "granite3.3:2b"
options = {
    "temperature": 0,
    "top-p": 0.9
}

In [3]:
def chat_assistant(question):
    messages = [
        {
            "role": "system",
            "content": """
            Você é um mecânico especialista em carros Hyundai HB20. 
            Sua tarefa é responder perguntas sobre a operacão, manutenção e especificação dos veículos Hyundai HB20 de forma objetiva e direta.
            Forneça respostas curtas e diretas.
            Ignore qualquer pergunta não relacionada ao veículos Hyundai HB20, respondendo de forma cordial 'Desculpe não tenho esta informacão'.
            Responda cordialmente 'Desculpe não tenho esta informacão' para perguntas que você não sabe a resposta.
            """
        },
        {
            "role": "user",
            "content": question
        } 
    ]

    resp = ollama.chat(model=model, messages=messages, options=options)
    return resp["message"]["content"]

In [4]:
print("> R:", chat_assistant("Qual a pressão em psi para carga normal dos pneus do meu HB20?"))

> R: A pressão recomendada de ar dos pneus do Hyundai HB20 é de 32 psi (PSI) para carga normal.


In [5]:
print("> R:", chat_assistant("Qual é tipo de bateria da chave inteligente do meu HB20?"))

> R: A Hyundai HB20, desde o modelo 2016 em diante, utiliza uma bateria de íon de lítio para a chave inteligente.


## RAG - Retrieval-Augmented Generation